In [1]:
import numpy as np
import glob
from distort_calibration import *
from cartesian import*
from registration_3d import*
from optical_tracking import*
from em_tracking import *
from eval import *
from pathlib import Path
import argparse

In [2]:
from scipy.interpolate import BPoly

In [3]:
# scale points to a box
# X is a n*3 numpy matrix qmin and qmax are float values stand for lower and upper bound
def scale_to_box( X, qmin, qmax ):

    scale = np.abs( qmax - qmin )
    
    scaled_X = ( X - qmin ) / scale  # normalized input
    
    return scaled_X, qmin, qmax

In [4]:
def generate_Bpoly_basis( order ):
    
    x_break = [0, 1]
    basis = []
    
    for i in range( order + 1 ):
        c =  np.zeros( (order + 1 , 1) )
        c[i] = 1
        basis.append( BPoly( c, x_break ) )

    return basis

In [5]:
def generate_berntensor( points , qmin, qmax, order ):
#Function to generatea tensor of the 3-D Bernstein functions where 
#F_ijk = b_i(x)*b_j(y)*b_k(z).

    bern_basis = generate_Bpoly_basis( order )
    
    scaled_points = scale_to_box( points , qmin, qmax )[0]

    scaled_x = scaled_points[:, 0].reshape( ( -1, 1 ) )
    scaled_y = scaled_points[:, 1].reshape( ( -1, 1 ) )
    scaled_z = scaled_points[:, 2].reshape( ( -1, 1 ) )
    
    bern_matrix = np.zeros( ( points.shape[0], ( order + 1 ) ** 3 ) ) # 3D
    
    for i in range( order + 1 ):
        for j in range( order + 1 ):
            for k in range( order + 1 ):
                
                #F_ijk dot product for each point since we are using *
                F_ijk = ( bern_basis[i]( scaled_x ) ) * ( bern_basis[j]( scaled_y ) ) * ( bern_basis[k]( scaled_z ) )
                #we need to change the shape from (k,1) to (k,)
                F_ijk = F_ijk.reshape(-1,)
                bern_matrix[:, i * ( order + 1 ) ** 2 + j * ( order + 1 ) + k] = F_ijk
    
    return bern_matrix

In [6]:
# X: The input parameters to be fit to Y
# Y: The output parameters to be fit from X
# order: The highest order we use in Bernstein polynomial
def distortion_correction( x, y, order):

    qmin = np.min( x )      
    qmax = np.max( x )
    #print(qmin)
    #print(qmax)
    Bern_Matrix = generate_berntensor( x, qmin, qmax, order )
    
    A = Bern_Matrix
    B = y
    
    # AX = B
    X = np.linalg.lstsq(A, B, rcond=None)[0]
    print("AX-B max error: ",np.max(A@X-B))
    return X, qmin, qmax

In [7]:
# def main():
if True:

# sample names from DATA
# pa1-debug-a-calbody
# pa1-unknown-a-calreadings
    runtype = 0
    run = 2
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
    type = ['debug', 'unknown']
    data_dir = "DATA/"
    pa = "pa2-"
    # first read in Calbody (Nd, Na, Nc)
    # then readings  (ND, NA, NC, nframes)
    
    # the last output determines whether to show the result in terminal
    em_pivot = em_tracking( data_dir ,pa , type[runtype] , letters[run] , output = 0)
    optical_pivot = optical_tracking( data_dir ,pa , type[runtype] , letters[run] , output = 0)

    tmp_ce = distort_calibration( data_dir ,pa ,type[runtype] , letters[run] ,output = 0)
    
    C_exp = tmp_ce[0]
    C = tmp_ce[3]
    
    Nc = tmp_ce[1]
    Nframes = tmp_ce[2]
    # print(optical_pivot.shape)
    ep = np.transpose(em_pivot)
    op = np.transpose(optical_pivot)
    if(pa == "pa1"):
        tmp = eval( data_dir , type[runtype] , letters[run] , C_exp, em_pivot, optical_pivot)
        str_tmp = [str(float(x)) for x in tmp]
        print("Average for difference of C_exp = " + str_tmp[0])
        print("Variance for difference of C_exp = " + str_tmp[1])
        print("Max for difference of C_exp = " + str_tmp[2])
        print("Min for difference of C_exp = " + str_tmp[3])

        print("Average for difference of em_pivot = " + str_tmp[4])
        print("Variance for difference of em_pivot = " + str_tmp[5])
        print("Max for difference of em_pivot = " + str_tmp[6])
        print("Min for difference of em_pivot = " + str_tmp[7])

        print("Average for difference of opt_pivot = " + str_tmp[8])
        print("Variance for difference of opt_pivot = " + str_tmp[9])
        print("Max for difference of opt_pivot = " + str_tmp[10])
        print("Min for difference of opt_pivot = " + str_tmp[11])
        output_name = data_dir
    # print(em_pivot, optical_pivot)
    # print(Nc, Nframes)

In [8]:
C_exp.shape

(3375, 3)

In [9]:
C.shape

(3375, 3)

In [10]:
coef, qmin, qmax = distortion_correction( C, C_exp, 5 ) # 5 is order

0.11466642644961667


In [11]:
coef.shape

(216, 3)